# Importing all moldules

In [1]:
import alpha_vantage
import pandas
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import datetime

# CLASS ScriptData

In [2]:
class ScriptData(dict):

    global libray 
    libray = dict()

    def __getitem__(self,key):
        #print(type(lb[key]))
        #print(libray[key])
        return libray[key]


    def __setitem__(self, i, o):
        pass
    
    
    
    def __contains__(self, o):
        for key in libray.keys():
            if key == o:
                return True
            
        return False
    
    # __init__ function
    # def __init__(self):
    #     self = Userdict()


    def fetch_intraday_data(self,script):
        key = 'XTP4AMN2N5H9BL51'
        ts = TimeSeries(key,output_format='pandas')
        global data ,meta 
        data, meta = ts.get_intraday(script,interval='60min',outputsize='full')


    def convert_intraday_data(self,script):
        columns=['open','high','low','close','volumn']
        data.columns = columns
        data['timestamp']=data.index
        data.reset_index(drop=True, inplace=True)
        #data['timestamp']=data.index
        libray[script]=data
        #self['GOOGL'] = data
        #print(self[script])




    

In [3]:
script_data = ScriptData()

In [4]:
script_data.fetch_intraday_data('GOOGL')

In [5]:
script_data.convert_intraday_data('GOOGL')

In [6]:
script_data['GOOGL']

,open,high,low,close,volumn,timestamp
0,91.6400,91.6500,91.5200,91.65,32061.0,2023-01-11 20:00:00
1,91.6500,91.6900,91.5400,91.64,15033.0,2023-01-11 19:00:00
2,91.5500,91.6487,91.5400,91.56,16227.0,2023-01-11 18:00:00
3,91.5200,91.6200,90.8768,91.60,905942.0,2023-01-11 17:00:00
4,90.8600,91.6000,90.8100,91.50,5789247.0,2023-01-11 16:00:00
...,...,...,...,...,...,...
596,98.2902,98.3500,97.5600,97.56,92811.0,2022-11-16 09:00:00
597,98.3900,98.3900,98.1200,98.27,12182.0,2022-11-16 08:00:00
598,98.2900,98.4900,97.9500,98.48,9722.0,2022-11-16 07:00:00
599,98.4300,98.6300,98.3000,98.34,6119.0,2022-11-16 06:00:00


In [7]:
script_data.fetch_intraday_data('AAPL')

In [8]:
script_data.convert_intraday_data('AAPL')

In [9]:
script_data['AAPL']

,open,high,low,close,volumn,timestamp
0,133.4500,133.4900,133.3900,133.4900,63150.0,2023-01-11 20:00:00
1,133.4400,133.4700,133.3600,133.4400,40088.0,2023-01-11 19:00:00
2,133.4200,133.4700,133.3500,133.4400,102116.0,2023-01-11 18:00:00
3,133.4900,133.5000,133.1400,133.4300,2703696.0,2023-01-11 17:00:00
4,132.5300,133.5100,132.4950,133.4800,11918004.0,2023-01-11 16:00:00
...,...,...,...,...,...,...
598,150.3100,151.0391,147.8131,148.8818,661384.0,2022-11-16 09:00:00
599,149.6208,150.0603,149.3811,149.9005,126362.0,2022-11-16 08:00:00
600,149.9105,149.9105,149.3112,149.6208,84849.0,2022-11-16 07:00:00
601,149.9305,150.3799,149.8506,149.9105,26134.0,2022-11-16 06:00:00


In [10]:
'GOOGL' in script_data

True

In [11]:
'AAPL' in script_data 

True

In [12]:
'UNCLE' in script_data

False

# Indicator method


In [50]:
def indicator1(df,timeperiod=10):
    df1 = df
    df1['indicator'] = df['close'].rolling(timeperiod).mean()
    data['indicator'] = df['indicator']
    return df1[['timestamp','indicator']].copy()

In [51]:
indicator1(script_data['GOOGL'], timeperiod=2)

,timestamp,indicator
0,2023-01-11 20:00:00,NaN
1,2023-01-11 19:00:00,91.645
2,2023-01-11 18:00:00,91.600
3,2023-01-11 17:00:00,91.580
4,2023-01-11 16:00:00,91.550
...,...,...
596,2022-11-16 09:00:00,98.055
597,2022-11-16 08:00:00,97.915
598,2022-11-16 07:00:00,98.375
599,2022-11-16 06:00:00,98.410


In [52]:
indicator1(script_data['AAPL'], 5)

,timestamp,indicator
0,2023-01-11 20:00:00,NaN
1,2023-01-11 19:00:00,NaN
2,2023-01-11 18:00:00,NaN
3,2023-01-11 17:00:00,NaN
4,2023-01-11 16:00:00,133.45600
...,...,...
598,2022-11-16 09:00:00,148.26358
599,2022-11-16 08:00:00,148.53124
600,2022-11-16 07:00:00,148.80688
601,2022-11-16 06:00:00,149.27928


In [257]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd


class Strategy(ScriptData):
    
    def __init__(self,script):
        self.script = script


    def get_script_data(self):
        self.fetch_intraday_data(self.script)
        self.convert_intraday_data(self.script)
        global df
        global indicator_data
        df = data['close'].to_frame()
        df['timestamp'] = data['timestamp']
        df['indicator_data'] = (indicator1(df,5))['indicator']

        

    def get_signals(self):
        
        signals = df
        signals['signal'] = np.nan
        for i in range(len(df)):
            if df.iloc[i-1]['close']>df.iloc[i-1]['indicator_data'] and df.iloc[i]['close'] < df.iloc[i]['indicator_data']:
                signals['signal'][i] = 'BUY'
            elif df.iloc[i-1]['close'] < df.iloc[i-1]['indicator_data'] and df.iloc[i]['close'] > df.iloc[i]['indicator_data']:
                signals['signal'][i] = 'SELL'
        signals = signals.dropna()
        signals.reset_index(inplace = True)
           # [['timestamp','signal']].copy().dropna()
        return signals[['timestamp','signal']].copy()
        
        
    def plot(self):
        
        
        
        df = data

        
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        # include candlestick with rangeselector
        fig.add_trace(go.Candlestick(x=df['timestamp'],
                        open=df['open'], high=df['high'],
                        low=df['low'], close=df['close'],name="Historical Data"),
                    secondary_y=True)

        # include a go.Bar trace for indicators
        fig.add_trace(go.Scatter(x=df['timestamp'], y=df['indicator'],name="SMA",line= {'color': 'rgb(1,10,1)', 'width': 0.5}),
                    secondary_y=False)
        fig.update_layout(
                yaxis=dict(),
                xaxis=dict(
                rangeselector=dict(
                    buttons=list([
                        dict(count=1,
                            label="1m",
                            step="month",
                            stepmode="todate"),
                        dict(count=6,
                            label="6m",
                            step="month",
                            stepmode="todate"),
                        dict(count=1,
                            label="YTD",
                            step="year",
                            stepmode="todate"),
                        dict(count=1,
                            label="1y",
                            step="year",
                            stepmode="todate"),
                        dict(step="all")
                    ])
                ),
                
                rangeslider=dict(
                    visible=False
                ),
            )
            )
        fig.layout.yaxis2.showgrid=False
        fig.show()

       
            
         
            
            
        
        
        


In [258]:
strategy = Strategy('NVDA')
strategy.get_script_data()

In [259]:
strategy.get_signals()

,timestamp,signal
0,2023-01-11 09:00:00,SELL
1,2023-01-10 19:00:00,BUY
2,2023-01-10 11:00:00,SELL
3,2023-01-10 10:00:00,BUY
4,2023-01-10 06:00:00,SELL
...,...,...
136,2022-11-16 17:00:00,SELL
137,2022-11-16 16:00:00,BUY
138,2022-11-16 15:00:00,SELL
139,2022-11-16 11:00:00,BUY


In [260]:
strategy.plot()